In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('miss').getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/shared_uploads/sejal@ibm.com/ContainsNull.csv', header=True, inferSchema=True)

In [0]:
df.show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# dropping missing data
df.na.drop().show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# with thresh param, row must have at least 2 non-null values in order to avoid being dropped
df.na.drop(thresh=2).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# only drop the row if all values are null
df.na.drop(how='all').show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| null|
emp2| null| null|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# we just care that sales are non-null
df.na.drop(subset=['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
df.printSchema()

root
-- Id: string (nullable = true)
-- Name: string (nullable = true)
-- Sales: double (nullable = true)

In [0]:
# filling missing values for string columns
df.na.fill('FILL VALUE').show()

+----+----------+-----+
 Id| Name|Sales|
+----+----------+-----+
emp1| John| null|
emp2|FILL VALUE| null|
emp3|FILL VALUE|345.0|
emp4| Cindy|456.0|
+----+----------+-----+

In [0]:
# filling missing values for numeric (double and int) columns
df.na.fill(0).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John| 0.0|
emp2| null| 0.0|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+

In [0]:
# fill missing values for specified columns
df.na.fill('No Name', subset=['Name']).show()

+----+-------+-----+
 Id| Name|Sales|
+----+-------+-----+
emp1| John| null|
emp2|No Name| null|
emp3|No Name|345.0|
emp4| Cindy|456.0|
+----+-------+-----+

In [0]:
# fill values with mean value for numeric column
from pyspark.sql.functions import mean
mean_val = df.select(mean(df['Sales'])).collect()[0][0]
df.na.fill(mean_val, ['Sales']).show()

+----+-----+-----+
 Id| Name|Sales|
+----+-----+-----+
emp1| John|400.5|
emp2| null|400.5|
emp3| null|345.0|
emp4|Cindy|456.0|
+----+-----+-----+